# Projeto Final - Controle de Pouso

###### Alunos: Paulo Bessa, Sarah Nadaud e Vitor Krauss.

###### Data: 8 de Julho de 2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import random
from scipy.interpolate import interp1d
import io
import base64
from IPython.display import HTML

### Objetivos:
 - Modelar a dinâmica de uma espaçonave em retorno à Terra, sujeita à ação da gravidade e de vento
 - Desenvolver uma inteligência artificial capaz de aterrisar a nave em um lugar adequado através do controle de um propulsor central e dois propulsores laterais, capazes apenas de rodar a nave em torno de seu centro de massa
 - Melhorar o algoritmo para minimizar a energia gasta com os combustíveis

### Hipóteses 

Vamos modelar o problema considerando, primeiramente, que:
 - a nave tem formato circular 
 - temos 3 propulsores, um maior na parte inferior da nave e que faz uma força em direção ao seu centro de massa, e dois na parte superior fazendo forças transversais, que rotacionam a nave em torno de seu centro de massa
 - o vento pode ser qualquer vetor em $\mathbb{R}^2$
 - vamos considerar $g = 9,8$ $ m/s^2$
 - o combustivel está uniformemente distribuido na nave , então conforme ele for gasto, a massa total irá diminuir mas o centro de massa continuará no centro do círculo
 - a posição inicial da nave é tem sua coordenada $x$ distribuida uniformemente em um determinado intervalo

### Modelagem
##### Quais forças estão agindo sobre a nave ? 

 - A força da gravidade  $F_{g}(t) = m(t) g$ , onde $m(t)$ é a massa total da nave no tempo $t$ e $g=9.8$ $m/s^2$ a aceleração gravitacional.  Essa força sempre tem direção constante $(0,-1)$ atuando no centro de massa. 
 - Os propulsores laterais exercerão forças tangenciais à nave, de forma a rotacioná-la em torno do seu centro de massa.
 - A força do vento : como o nosso objeto é uma esfera, a força do vento não rotaciona a nave, porém causa uma aceleração da nave em si. A força resultante do vento sobre a nave é dada por $ F_{vento} = A * P * C $, onde A é a área transversal da nave, $P = 0,613 V^2$ a pressão do vento, $V$ a velocidade do vento em metros por segundo e $C=0.1$ o coeficiente de arrasto de uma esfera lisa no ar.
 
##### Mecanismos da Nave

 - A nave conta com um propulsor principal, que exerce uma força $F_p$ na direção do centro de massa da nave.
 - A nave conta com dois propulsores laterais, que exercem forças tangenciais à nave, de forma a rotacioná-la em torno do seu centro de massa.
 - A nave descreve um ângulo $\theta$ com a vertical, de forma que o vetor da força do propulsor principal descreve o mesmo ângulo com a vertical. Ou seja, $\theta$ determina a direção da força da nave. Usaremos os propulsores laterais para ajustar este ângulo para um valor adequado em cada instante.

##### Algoritmo

A cada instante a nave pode acionar seus propulsores para ajustar sua posição. O objetivo é desenvolver uma estratégia para nave de forma a utilizar a menor quantidade de combustível possível.

No instante $t$, a forma com que determinamos o devido uso do propulsor é a seguinte:

No instante $t$, a nave está na posição $(x_t, y_t)$. A nave tem conhecimento dos seus possíveis locais de pouso. Para cada um destes locais $(x_p, y_p)$, podemos calcular o tempo $T_p$ que a nave levaria para atingir a altura $y_p$ somente sob ação da gravidade, da seguinte forma: 

Temos que
$$ y_p = y_t + v_tT_p + \frac{g}{2} T_p^2$$
Dai, basta resolver a equação do segundo grau acima e pegar a raíz positiva.

Uma vez que determinamos $T_p$, e considerando que neste instante $t$ a nave tem conhecimento do  vetor vento relativo $F_v$. Supondo que o vento relativo será constante daqui para a frente, podemos fazer uma projeção da trajetória da nave segundo este vento, ou seja, podemos calcular a coordenada $x_{T_p}$ que a nave atingiria no tempo $T_p$(e altura $y_p$). Feito isso, calculamos a distância $x_{T_p} - x_p$. Calculadas estas distâncias para cada possível local de pouso $p$, a nave escolhe como alvo para o pouso o ponto $p^{*}$ que minimiza a distância $x_{T_p} - x_p$.

Agora que temos $T_{p^{*}}$, calculamos as acelerações vertical $a_y$ e horizontal $a_x$ necessárias para, no instante $T_{p^{*}}$, a nave atingir a posição $(x_{p^{*}}, y_{p^{*}})$, com velocidade vertical final $V_{y}^{f} = 0$. Para calcular estas acelerações, usamos que: 

$$ V_{y}^{f} = V_{y}^{t} + \int_{0}^{T_{p^{*}}} (a_y - g)ds $$

o que nos dá 

$$ a_y = \frac{-V_{y}^{t}}{T_{p^{*}}} + g$$

e 

$$ x_{p^{*}} = x_t + V_{x}^{t}T_{p^{*}} + \frac{a_x + a_v}{2} T_{p^{*}}^2 $$ 

o que nos dá 

$$ a_x = \frac{2(x_{p^{*}} - x_t -  V_{x}^{t}T_{p^{*}})}{T_{p^{*}}^2} - a_v$$

Agora que temos $a_x$ e $a_y$, calculamos o vetor $F_{propulsor} = \sqrt{a_x^2 + a_y^2}*(a_x, a_y) = \sqrt{a_x^2 + a_y^2}*M(-\gamma)*(0,1)$, onde $M(-\gamma)$ é matriz de rotação(no sentido horário) relativa ao ângulo $\gamma$, o ângulo que o vetor $F_{propulsor}$ descreve com a vertical. Este ângulo $\gamma$ é então o valor desejado para o ângulo $\theta$ da nave. Para que a nave descreva este ângulo $\gamma$, i.e, para que $\theta = \gamma$, usamos os propulsores laterais para conferir uma aceleração ângular à nave. Para calcular a devida aceleração ângular $\alpha$, já levamos em conta que a simulação é discretizada no computador. Assim, seja $2N$ o número de iterações na qual queremos sair do ângulo $\theta_t$ para $\gamma$. Queremos $\alpha$ tal que nas próximas $N$ iterações daremos uma aceleração ângular $\alpha$, e depois, nas restantes $N$ iterações, daremos aceleração $-\alpha$, de forma a atingir o ângulo $\gamma$ com velocidade ângular $0$. Assim, 

$$ \gamma = \theta_t + \sum_{i=1}^{N} (V_{\theta}^{t} + \alpha*i) + \sum_{i=1}^{N} (V_{\theta}^{t} + \alpha*(N-i))  $$

Resolvendo a equação acima para $\alpha$, obtemos:

$$ \alpha = \frac{\gamma - \theta_t - 2N*V_{\theta}^{t}}{N^2} $$

Quando a nave está uma certa distância vertical pequena, fazemos que $\gamma$ seja obrigatoriamente igual a zero, para que no momento do pouso a nave tenha ângulo $\theta = 0$, isto é, ela pousa com o propulsor principal apontando para baixo, e com velocidade ângular muito próxima de zero. 

In [2]:
class Nave:
    "classe que implementa toda a simulação do jogo, e não apenas da nave. Ou seja, o nome é um pouco indevido"
    def __init__(self):
        "metodo construtor, define constantes e especificações da nave, além de chamar os métodos que simulam o seu movimento."
        self.g = -9.8
        self.m = 1*(10**4)
        self.k = 0.5 
        self.area = 400
        self.c = 0.1    #coeficiente de arrasto 
        self.vento_c = np.random.uniform(5, 20)
        self.vento = self.vento_c
        self.theta = 0.0  #angulo que a nave descreve com a vertical
        self.r = 40  #raio
        self.F_gravidade = self.m*self.g*np.array([0,1])
        self.F_propulsor = np.array([0,0])
        self.ax = 0   #acelerações iniciais
        self.ay = 0
        self.F_vento = self.area*(0.613*(self.vento**2))*self.c*np.array([1,0])
        self.x = np.random.uniform(200,800)   #coordenada x inicial, aleatoria
        self.y = 5*(10**3)                    #coordenada y inicial
        self.rot = self.matrix(self.theta)   #matriz de rotação
        self.vx = 0.0                        #velocidade inicial em x
        self.vy = np.random.uniform(-100, 0) #Velocidade inicial em y, aleatoria
        self.vtheta = 0.0     #velocidade angular 
        self.ac_ang = 0   #aceleração angular
        self.j = 0   #contador
        self.p = [0,0]
        self.make_ground()
        self.pos = self.positions()      
        
    def matrix(self, alpha):
        "retorna a matriz de rotação por um angulo alpha"
        m = np.full((2,2), 0.0)
        m[0][0] = np.cos(alpha)
        m[0][1] = -1*np.sin(alpha)
        m[1][0] = np.sin(alpha)
        m[1][1] = m[0][0]
        return m
    
    def make_ground(self):
        "cria o chão, de maneira aleatória"
        M = 10000
        N = random.randint(30, 40)
        xs = [None]*N
        self.x_to_plot = []
        self.y_to_plot = []
        is_cons = np.random.binomial(1, 0.3, N)
        self.ground = []
        xs[0] = 0
        xs[-1] = 1000
        xs[1:-1] = np.sort(np.random.uniform(0, 1000, N-2))
        y = np.random.uniform(350, 1000)
        self.sites = []
        for i in range(N-1):
            if(is_cons[i] == 1):
                temp = y
                self.sites.append([(xs[i+1]+xs[i])/2, y])
            else: temp = np.random.uniform(350, 1000)
            f = interp1d([xs[i], xs[i+1]], [y, temp])
            x = np.linspace(xs[i], xs[i+1], M)
            y = f(x)
            self.ground = self.ground + [[x[j], y[j]] for j in range(M)]
            self.x_to_plot = self.x_to_plot + list(x)
            self.y_to_plot = self.y_to_plot + list(y)
            y = temp
            
    def landed(self):
        "método que verifica se a nave já aterrisou ou não"
        for i in range(len(self.ground)):
            p = self.ground[i]
            if( abs(self.x-p[0]) < 25 and abs(self.y - p[1]) < 35 ): return True
    
    def movimento(self):
        "realiza a movimentação da nave"
        alpha = 0.05 # coeficiente para discretização, de forma a torna-la "mais contínua"
        self.vento = self.vento_c*np.sin(self.k)
        self.F_vento = self.area*(0.613*(self.vento**2))*self.c*np.array([np.cos(self.k), np.sin(self.k)])
        self.rot = self.matrix(-1*self.theta)
        F_resul = self.F_gravidade + self.F_vento + np.dot(self.rot, self.F_propulsor)
        ax_resul = F_resul[0]/self.m 
        ay_resul = F_resul[1]/self.m
        #if(abs(self.y - self.p[1]) < 150): beta = 0.05
        #else: beta = 1
        self.vx = self.vx + alpha*ax_resul
        self.vy = self.vy + alpha*ay_resul
        self.vtheta = self.vtheta + self.ac_ang
        #guardando os dados anteriores
        tempx = self.x
        tempy = self.y
        temp_theta = self.theta  
        #atualizando os dados
        self.x = self.x + alpha*self.vx
        self.y = self.y + alpha*self.vy
        self.theta = self.theta + self.vtheta
        self.j = self.j + 1
        self.k = self.k + 0.001
        d = np.sqrt((tempx - self.x)**2 + (tempy - self.y)**2) #distancia percorrida
        tau = abs(self.ac_ang * self.m * (self.r)/4)  #torque
        W = abs(d*np.sqrt(self.F_propulsor[0]**2 + self.F_propulsor[1]**2) + tau*self.r*abs(temp_theta - self.theta)) #trabalho realizado entre as iterações
        self.m = self.m - W/(43*(10**6))  #massa dinamica , pela gasto de combustivel 
        self.find_closest()

    def positions(self):
        ""
        L = 2500
        positions = [None]*(L+1)
        self.vs = [None]*(L+1)
        self.Fvs = [None]*(L+1)
        self.ps = [None]*(L+1)
        self.ths = [None]*(L+1)
        self.vxs = [None]*(L+1)
        self.vys = [None]*(L+1)
        self.vths = [None]*(L+1)
        self.ms = [None]*(L+1)
        i = 0
        while(self.y > 0):
            if(i > L): break
            if(self.landed() == True): 
                if(abs(self.x-self.p[0]) < 30 and abs(self.y - self.p[1]) < 30 and abs(self.vy) < 10 and abs(self.theta) < 0.1 and abs(self.vtheta) < 0.1):
                    self.success = True
                else: self.success = False
                break
            positions[i] = (self.x, self.y)
            self.vs[i] = self.vento
            self.ths[i] = self.theta
            self.Fvs[i] = self.F_vento
            self.ps[i] = self.p
            self.vxs[i] = self.vx
            self.vys[i] = self.vy
            self.vths[i] = self.vtheta
            self.ms[i] = self.m
            self.movimento()
            i += 1
        return list(filter(None, positions))
    
    def find_closest(self):
        "acha o local de pouso adequado mais perto e calcula as acelerações necessarias para atingi-lo"
        temp = 10**10
        time = 1
        index = 0
        for i in range(len(self.sites)):
            t = max(np.roots([self.g/2, self.vy, self.y - self.sites[i][1]]))  #tempo que a nave levaria para atingir o chão
            d = abs(self.x + self.vx*t + ((self.F_vento[0]/self.m)/2)*(t**2) - self.sites[i][0]) #distancia horizontal entre a projeção e o possivel local de pouso
            if(d < temp):
                temp = d
                time = t
                index = i
        if(time.imag == 0):
            time = time.real
            p = self.sites[index]  #escolhe o local de pouso que minimiza a distancia
            self.p = p
            self.ax = (2/(time**2))*(p[0]-self.x-self.vx*time) - self.F_vento[0]/self.m #acceleração desejada pro x
            self.ay = -1*self.vy/time + self.g  #acceleração desejada pro y
            a = np.sqrt(self.ax**2 + self.ay**2) 
            gamma = np.arctan(self.ax/self.ay) #angulo desejado 
            self.F_propulsor = self.m*a*np.array([0,1])
            L = 4
            if(abs(self.y - self.p[1]) < 130):  #quando muito proximo do chao, voltar o angulo para zero e com velocidade 0
                gamma = 0
            if(self.j % L <= L/2):   #calculando a aceleração angular necessaria para chegar no angulo desejado
                self.ac_ang = (gamma - self.theta - 2*L*self.vtheta)/(L**2)
            else:        
                self.ac_ang = -1*(gamma - self.theta - 2*L*self.vtheta)/(L**2)

In [3]:
def anima(nave, name):
    "faz da animação da simulação"
    pos = nave.pos
    fig, ax = plt.subplots(1, figsize=(4, 4))
    sinc, = ax.plot(pos[0][0], pos[0][1], 'bo')
    x_land_sites = [x[0] for x in nave.sites]
    y_land_sites = [x[1] for x in nave.sites]
    
    def animate(i):
        ax.clear()
        ax.set_facecolor('#00004d')
        ax.set_ylim(0, pos[0][1])
        ax.set_xlim(0, 1000)
        ax.plot(pos[i][0], pos[i][1], 'bo', markersize=9, color='#ff99e6')
        ax.plot(x_land_sites, y_land_sites, 'x')
        ax.plot(nave.ps[i][0], nave.ps[i][1], 'x', color='#ff3399')
        ax.fill_between(nave.x_to_plot, np.zeros(len(nave.x_to_plot)), nave.y_to_plot, color='#ffffb3')
        v = nave.Fvs[i]
        n = np.sqrt(v[0]**2 + v[1]**2)
        w = v/n
        ax.quiver([50], [4.6*(10**3)], [w[0]], [w[1]], color='#66ff66')
        ax.text(10, 25, "Vento = %1.3f" %(nave.vs[i]), color='#003399', fontsize = 8)
        ax.text(10, 200, "Theta = %1.3f" %(nave.ths[i]), color='#003399', fontsize = 8)
        ax.text(720, 25, "Vx = %1.3f" %(nave.vxs[i]), color='#003399', fontsize = 8)
        ax.text(720, 200, "Vy = %1.1f" %(nave.vys[i]), color='#003399', fontsize = 8)
        ax.text(340, 200, "Vtheta = %1.4f" %(nave.vths[i]), color='#003399', fontsize = 8)
        ax.text(340, 25, "Massa = %1.4f" %(nave.ms[i]-7*(10**3)), color='#003399', fontsize = 8)
        ax.set_title("Controle de Pouso") 
        
    def init():
        return sinc,
    
    N_FRAMES = len(pos)
    ani = animation.FuncAnimation(fig, animate, N_FRAMES, init_func=init, interval=10)
    ani.save(name+'.mp4', 'ffmpeg')

In [4]:
nave = Nave()

In [5]:
anima(nave, 'animacao')

In [6]:
video = io.open('animacao', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

#### Considerações finais
Para uma primeira modelagem do problema ousamos dizer que os resultados foram bons. Quanto à parte da modelagem física do problema nossa simulação não apresentou falhas muito significativas e consideramos que a modelagem foi boa, considerando-se as limitações de um computador, cujo meio é discreto, para um sistema contínuo. Ao nosso ver, a parte da inteligência artificial da nave também apresentou, em geral, um resultado satisfatório. Contudo, apontamos aqui alguns pontos em que tivemos dificuldade e que, dados mais tempo e dedicação, podem, e devem, ser melhorados: 

###### Melhorias a serem feitas no futuro:
 - Levar em consideração as montanhas no meio do caminho: nossa nave não é capaz de identificar obstáculos em seu caminho, o que, eventualmente, leva a uma colisão.
 - Buscamos implementar um método que verifica não apenas se a nave atingiu o chão, mas se ela aterrisou no local desejado e com ângulo e velocidades vertical e ângular seguras para o pouso. Neste momento, porém, nossa nave ainda não está pousando com a velocidade vertical segura, que consideramos ser abaixo de $10m/s$. Acreditamos que a origem do problema está em erro numérico que ocorre quando a distância vertical da nave até o chão é muito pequena, o que nos dá um $T_p$ muito próximo de zero, e depois fazemos algumas divisões com esse $T_p$. Seria interessante corrigir este erro para estas distâncias pequenas. Por enquanto, por causa desse problema na velocidade, nosso método dirá que a nave não aterrisou com sucesso.
 - Explorar outras estratégias de uso do combustivel